In [16]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import norm, t
from scipy.optimize import fsolve, minimize
import inspect
import risk_mgm_lib as rml
from functools import partial
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

## Problem 1

In [129]:
current_date = datetime(2022, 3, 13)
expire_date = datetime(2022, 4, 15)
T = (expire_date - current_date).days / 365

S = 151.03
X = 165
sigma = 0.2

r = 0.0425
q = 0.0053
b = r - q
     

In [130]:
# Calculate the normal cdf's for black scholes
def calculate_d1(S, X, T, sigma, b):
    return (np.log(S / X) + (b + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))

def calculate_d2(d1, T, sigma):
    return d1 - sigma * np.sqrt(T)
     

In [131]:
# calculate the greeks
def gbsm_delta(option_type, S, X, T, sigma, r, b):
    is_call = 1 if option_type == "Call" else -1
    d1 = calculate_d1(S, X, T, sigma, b)
    delta = norm.cdf(d1 * is_call, 0, 1) * is_call
    return delta

def gbsm_gamma(option_type, S, X, T, sigma, r, b):
    d1 = calculate_d1(S, X, T, sigma, b)
    d2 = calculate_d2(d1, T, sigma)
    gamma = norm.pdf(d1, 0, 1) / (S * sigma * np.sqrt(T))
    return gamma

def gbsm_vega(option_type, S, X, T, sigma, r, b):
    d1 = calculate_d1(S, X, T, sigma, b)
    d2 = calculate_d2(d1, T, sigma)
    vega = S * norm.pdf(d1, 0, 1) * np.sqrt(T)
    return vega

def gbsm_theta(option_type, S, X, T, sigma, r, b):
    is_call = 1 if option_type == "Call" else -1
    d1 = calculate_d1(S, X, T, sigma, b)
    d2 = calculate_d2(d1, T, sigma)
    theta = -S * np.exp((b - r) * T) * norm.pdf(d1, 0, 1) * sigma / (2 * np.sqrt(T)) \
            -(b - r) * S * np.exp((b - r) * T) * norm.cdf(d1 * is_call, 0, 1) * is_call \
            -r * X * np.exp(-r * T) * norm.cdf(d2 * is_call, 0, 1) * is_call
    return theta

def gbsm_rho(option_type, S, X, T, sigma, r, b):
    is_call = 1 if option_type == "Call" else -1
    d1 = calculate_d1(S, X, T, sigma, b)
    d2 = calculate_d2(d1, T, sigma)
    rho = X * T * np.exp(-r * T) * norm.cdf(d2 * is_call, 0, 1) * is_call \
          - is_call * T * S * np.exp((b-r)*T) * norm.cdf(is_call * d1)
    return rho

def gbsm_carry_rho(option_type, S, X, T, sigma, r, b):
    is_call = 1 if option_type == "Call" else -1
    d1 = calculate_d1(S, X, T, sigma, b)
    d2 = calculate_d2(d1, T, sigma)
    carry_rho = S * T * np.exp((b - r) * T) * norm.cdf(d1 * is_call, 0, 1) * is_call
    return carry_rho
     

In [126]:
# calculate first order derivative
def first_order_der(func, x, delta):
    return (func(x + delta) - func(x - delta)) / (2 * delta)

# calculate second order derivative
def second_order_der(func, x, delta):
    return (func(x + delta) + func(x - delta) - 2 * func(x)) / delta ** 2

def cal_partial_derivative(func, order, arg_name, delta=1e-3):
  # initialize for argument names and order
    arg_names = list(inspect.signature(func).parameters.keys())
    derivative_fs = {1: first_order_der, 2: second_order_der}

    def partial_derivative(*args, **kwargs):
        # parse argument names and order
        args_dict = dict(list(zip(arg_names, args)) + list(kwargs.items()))
        arg_val = args_dict.pop(arg_name)

        def partial_f(x):
            p_kwargs = {arg_name:x, **args_dict}
            return func(**p_kwargs)
        return derivative_fs[order](partial_f, arg_val, delta)
    return partial_derivative
     

In [132]:

def gbsm(option_type, S, X, T, sigma, r, b):
    d1 = (np.log(S / X) + (b + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    is_call = 1 if option_type == "Call" else -1

    res = is_call * (S * np.e ** ((b - r) * T) * \
                   norm(0, 1).cdf(is_call * d1) \
                   - X * np.e ** (-r * T) * \
                   norm(0, 1).cdf(is_call * d2))
    
    return res

In [133]:
# delta
delta_call = gbsm_delta("Call", S, X, T, sigma, r, b)
delta_put = gbsm_delta("Put", S, X, T, sigma, r, b)
gbsm_delta_num = cal_partial_derivative(gbsm, 1, 'S')
delta_call_num = gbsm_delta_num("Call", S, X, T, sigma, r, b)
delta_put_num = gbsm_delta_num("Put", S, X, T, sigma, r, b)
print(delta_call, delta_put)
print(delta_call_num, delta_put_num)

0.08301107089626869 -0.9169889291037313
0.08297130374668171 -0.9165496329472944


In [134]:
# gamma
gamma_call = gbsm_gamma("Call", S, X, T, sigma, r, b)
gamma_put = gbsm_gamma("Put", S, X, T, sigma, r, b)
gbsm_gamma_num = cal_partial_derivative(gbsm, 2, 'S')
gamma_call_num = gbsm_gamma_num("Call", S, X, T, sigma, r, b)
gamma_put_num = gbsm_gamma_num("Put", S, X, T, sigma, r, b)
print(gamma_call, gamma_put)
print(gamma_call_num, gamma_put_num)

0.016830979206204362 0.016830979206204362
0.016822911064195978 0.016822951920403284


In [135]:
# vega
vega_call = gbsm_vega("Call", S, X, T, sigma, r, b)
vega_put = gbsm_vega("Put", S, X, T, sigma, r, b)
gbsm_vega_num = cal_partial_derivative(gbsm, 1, 'sigma')
vega_call_num = gbsm_vega_num("Call", S, X, T, sigma, r, b)
vega_put_num = gbsm_vega_num("Put", S, X, T, sigma, r, b)
print(vega_call, vega_put)
print(vega_call_num, vega_put_num)

6.942036604441163 6.942036604441163
6.938653056250743 6.93865305626673


In [136]:
# theta
theta_call = gbsm_theta("Call", S, X, T, sigma, r, b)
theta_put = gbsm_theta("Put", S, X, T, sigma, r, b)
gbsm_theta_num = cal_partial_derivative(gbsm, 1, 'T')
theta_call_num = -gbsm_theta_num("Call", S, X, T, sigma, r, b)
theta_put_num = -gbsm_theta_num("Put", S, X, T, sigma, r, b)
print(theta_call, theta_put)
print(theta_call_num, theta_put_num)

-8.126522359668838 -1.9409914783019566
-8.126308803761084 -1.9407779203106656


In [137]:
# rho
rho_call = gbsm_rho("Call", S, X, T, sigma, r, b)
rho_put = gbsm_rho("Put", S, X, T, sigma, r, b)
gbsm_rho_num = cal_partial_derivative(gbsm, 1, 'r')
rho_call_num = gbsm_rho_num("Call", S, X, T, sigma, r, b)
rho_put_num = gbsm_rho_num("Put", S, X, T, sigma, r, b)
print(rho_call, rho_put)
print(rho_call_num, rho_put_num)

-0.030359909374904293 -1.2427313221864171
-0.030359909416688424 -1.2427313238703164


In [138]:
# carry rho
carry_rho_call = gbsm_carry_rho("Call", S, X, T, sigma, r, b)
carry_rho_put = gbsm_carry_rho("Put", S, X, T, sigma, r, b)
gbsm_carry_rho_num = cal_partial_derivative(gbsm, 1, 'b')
carry_rho_call_num = gbsm_carry_rho_num("Call", S, X, T, sigma, r, b)
carry_rho_put_num = gbsm_carry_rho_num("Put", S, X, T, sigma, r, b)
print(carry_rho_call, carry_rho_put)
print(carry_rho_call_num, carry_rho_put_num)

1.132953825011723 -12.515271800549371
1.1329550097096686 -12.515270634423814


### Binomial tree

In [139]:
def n_nodes(N):
    return (N + 2) * (N + 1) // 2

def node_index(i, j):
    return n_nodes(j - 1) + i

def binomial_tree_no_div(option_type, S0, X, T, sigma, r, N):
    is_call = 1 if option_type == "Call" else -1
    dt = T / N
    disc = np.exp(-r * dt)
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    C = np.empty(n_nodes(N), dtype=float)
            
    for i in np.arange(N, -1, -1):
        for j in range(i, -1, -1):
            S = S0 * u ** j * d ** (i - j)
            index = node_index(j, i)
            C[index] = max(0, (S - X) * is_call)
            if i < N:
                val = disc * (p * C[node_index(j + 1, i + 1)] + (1 - p) * C[node_index(j, i + 1)])
                C[index] = max(C[index], val)
                
    return C[0]

def binomial_tree(option_type, S0, X, T, div_time, div, sigma, r, N):
    if div_date is None or div is None:
        return binomial_tree_no_div(option_type, S0, X, T, sigma, r, N)
  
    is_call = 1 if option_type == "Call" else -1
    dt = T / N
    disc = np.exp(-r * dt)
    
    #calculate u, d, and p
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    new_T = T - div_time * dt
    new_N = N - div_time

    C = np.empty(n_nodes(div_time), dtype=float)
    for i in range(div_time, -1, -1):
        for j in range(i, -1, -1):
            S = S0 * u ** j * d ** (i - j)
            val_exe = max(0, (S - X) * is_call)
            if i < div_time:
                val = disc * (p * C[node_index(j + 1, i + 1)] + (1 - p) * C[node_index(j, i + 1)])
            else:
                val = binomial_tree(option_type, S - div, X, new_T, None, None, sigma, r, new_N)
            C[node_index(j, i)] = max(val_exe, val)
    
    return C[0]

In [140]:
# Assume N is 200
N = 200
value_no_div_call = binomial_tree_no_div("Call", S, X, T, sigma, r, N)
value_no_div_put = binomial_tree_no_div("Put", S, X, T, sigma, r, N)
print("Binomial tree value without dividend for call: " + str(value_no_div_call))
print("Binomial tree value without dividend for put: " + str(value_no_div_put))

Binomial tree value without dividend for call: 0.3420415058233237
Binomial tree value without dividend for put: 14.02022659787544


In [141]:
div_date = datetime(2022, 4, 11)
div = 0.88
div_time = int((div_date - current_date).days / (expire_date - current_date).days * N)

value_call = binomial_tree("Call", S, X, T, div_time, div, sigma, r, N)
value_put = binomial_tree("Put", S, X, T, div_time, div, sigma, r, N)
print("Binomial tree value with dividend for call: " + str(value_call))
print("Binomial tree value with dividend for put: " + str(value_put))

Binomial tree value with dividend for call: 0.2981599372927687
Binomial tree value with dividend for put: 14.55911431446306


In [142]:
# delta
cal_amr_delta_num = cal_partial_derivative(binomial_tree, 1, 'S0')
delta_call_amr = cal_amr_delta_num("Call", S, X, T, div_time, div, sigma, r, N)
delta_put_amr = cal_amr_delta_num("Put", S, X, T, div_time, div, sigma, r, N)
print(delta_call_amr, delta_put_amr)

0.0694035170404339 -0.9384266902472405


In [143]:
# gamma
cal_amr_gamma_num = cal_partial_derivative(binomial_tree, 2, 'S0', delta=1)
gamma_call_amr = cal_amr_gamma_num("Call", S, X, T, div_time, div, sigma, r, N)
gamma_put_amr = cal_amr_gamma_num("Put", S, X, T, div_time, div, sigma, r, N)
print(gamma_call_amr, gamma_put_amr)

0.0188730869562459 0.017693002005984226


In [144]:
# vega
cal_amr_vega_num = cal_partial_derivative(binomial_tree, 1, 'sigma')
vega_call_amr = cal_amr_vega_num("Call", S, X, T, div_time, div, sigma, r, N)
vega_put_amr = cal_amr_vega_num("Put", S, X, T, div_time, div, sigma, r, N)
print(vega_call_amr, vega_put_amr)

6.143196715376997 5.664125621646754


In [145]:
# theta
cal_amr_theta_num = cal_partial_derivative(binomial_tree, 1, 'T')
theta_call_amr = -cal_amr_theta_num("Call", S, X, T, div_time, div, sigma, r, N)
theta_put_amr = -cal_amr_theta_num("Put", S, X, T, div_time, div, sigma, r, N)
print(theta_call_amr, theta_put_amr)

-7.2765273123804315 -0.46564521876213405


In [146]:
# rho
cal_amr_rho_num = cal_partial_derivative(binomial_tree, 1, 'r')
rho_call_amr = cal_amr_rho_num("Call", S, X, T, div_time, div, sigma, r, N)
rho_put_amr = cal_amr_rho_num("Put", S, X, T, div_time, div, sigma, r, N)
print(rho_call_amr, rho_put_amr)

0.9426794754235357 -12.407586180172459


In [147]:
# sensitivity to change in dividend amount
# change the dividend amount on the first ex-dividend date by 1e-3
delta = 1e-3
call_value1 = binomial_tree("Call", S, X, T, div_time, div + delta, sigma, r, N)    
call_value2 = binomial_tree("Call", S, X, T, div_time, div - delta, sigma, r, N)    
call_sens_to_div_amount = (call_value1 - call_value2) / (2*delta)

put_value1 = binomial_tree("Put", S, X, T, div_time, div + delta, sigma, r, N)    
put_value2 = binomial_tree("Put", S, X, T, div_time, div - delta, sigma, r, N)    
put_sens_to_div_amount = (put_value1 - put_value2) / (2*delta)
print(f"Sensitivity to dividend amount: Call: {call_sens_to_div_amount:.3f}, Put: {put_sens_to_div_amount:.3f}")
     

Sensitivity to dividend amount: Call: -0.025, Put: 0.941


## Problem 2

In [342]:
np.random.seed(545)

In [343]:
current_date = datetime(2023, 3, 3)
div_date = datetime(2023, 3, 15)

def implied_vol_american(option_type, S0, X, T, div_time, div, r, N, market_price, x0=0.5):
    def equation(sigma):
        return binomial_tree(option_type, S0, X, T, div_time, div, sigma, r, N) - market_price
    # Back solve the binomial tree valuation to get the implied volatility
    return fsolve(equation, x0=x0, xtol=0.00001)[0]

def calculate_sim_values(portfolios, sim_prices, days_ahead=0):
    sim_values = pd.DataFrame(index=portfolios.index, 
                            columns=list(range(sim_prices.shape[0])))
    sim_prices = np.array(sim_prices)
    for i in portfolios.index:
        if portfolios["Type"][i] == "Stock":
            # For stock, the single value is its price
            single_values = sim_prices
        else:
            # For option, calculate values with gbsm method
            option_type = portfolios["OptionType"][i]
            X = portfolios["Strike"][i]
            T = ((portfolios["ExpirationDate"][i] - current_date).days - days_ahead) / 365
            sigma = portfolios["ImpliedVol"][i]
            div_time = int((div_date - current_date).days / (portfolios["ExpirationDate"][i] - current_date).days * N)
            div = 1
            option_values = []
            for S in sim_prices:
                option_values.append(binomial_tree(option_type, S, X, T, div_time, div, sigma, r, N))
            single_values = np.array(option_values)
    
        # Calculate the total values based on holding
        sim_values.loc[i, :] = portfolios["Holding"][i] * single_values
  
    # Combine the values for same portfolios
    sim_values['Portfolio'] = portfolios['Portfolio']
    return sim_values

In [344]:
portfolios = pd.read_csv('problem2.csv', parse_dates=['ExpirationDate'])
portfolios['CurrentValue'] = portfolios['CurrentPrice'] * portfolios['Holding']

S = 165
N = 25
current_date = datetime(2023, 3, 3)
div_date = datetime(2023, 3, 15)
r = 0.0425
div = 1

# Calculate the implied volatility for all portfolios
implied_vols = []
for i in range(len(portfolios.index)):
    if portfolios["Type"][i] == "Stock":
        implied_vols.append(None)
    else:
        option_type = portfolios["OptionType"][i]
        X = portfolios["Strike"][i]
        T = (portfolios["ExpirationDate"][i] - current_date).days / 365
        div_time = int((div_date - current_date).days / (portfolios["ExpirationDate"][i] - current_date).days * N)
        market_price = portfolios["CurrentPrice"][i]
        sigma = implied_vol_american(option_type, S, X, T, div_time, div, r, N, market_price)
        implied_vols.append(sigma)

# Store the implied volatility in portfolios
portfolios["ImpliedVol"] = implied_vols

In [345]:
S = 165
N = 25
current_date = datetime(2023, 3, 3)
div_date = datetime(2023, 3, 15)
r = 0.0425
div = 1

all_returns = pd.read_csv("DailyReturn.csv")

# Simulate the prices based on returns with normal distribution
std = all_returns['AAPL'].std()
np.random.seed(0)
sim_returns = norm(0, std).rvs((10, 100))
sim_prices = S * (1 + sim_returns).prod(axis=0)

# Calculate the current values and sim values
portfolios["CurrentValue"] = portfolios["CurrentPrice"] * portfolios["Holding"]
curr_values = portfolios.groupby('Portfolio')['CurrentValue'].sum()
sim_values = calculate_sim_values(portfolios, sim_prices, 10)

indices = list(np.linspace(0, len(portfolios['Portfolio'].unique())-1, len(portfolios['Portfolio'].unique())))
indices = list(map(int, indices))
sim_results = pd.DataFrame(index = indices, columns = list(range(len(sim_prices))))
port_names = sim_values['Portfolio'].unique()
for i in range(len(port_names)):
    sim_results.loc[i, :] = sim_values[sim_values['Portfolio'] == port_names[i]].sum()
    
sim_results['Portfolio'] = portfolios['Portfolio'].unique()
sim_results.index = sim_results['Portfolio']
sim_results = sim_results.loc[:, sim_results.columns != 'Portfolio']
cols = list(sim_results.index)
sim_results = sim_results.T
sim_results.columns = cols
sim_value_changes = pd.DataFrame(index = curr_values.index, columns = list(range(len(sim_prices)))).T
sim_value_changes.columns = cols
for port in sim_value_changes.columns:
    sim_vals = sim_results[port].values
    sim_value_changes[port] = sim_vals - curr_values[port]


# Calculate the value difference
# sim_value_changes = (sim_values.T - curr_values).T

In [346]:
sim_value_changes

,Straddle,SynLong,CallSpread,PutSpread,Stock,Call,Put,CoveredCall,ProtectedPut
0,4.410605,3.796681,5.399107,0.234535,9.710252,4.103643,0.306962,7.851163,9.771885
1,4.411091,3.797598,5.399107,0.234448,9.710954,4.104344,0.306747,7.851163,9.772457
2,9.928796,13.199748,-7052.315434,-0.842749,17.170882,11.564272,-1.635476,7.851163,15.863723
3,18.6121,25.158903,-0.255552,-1.839368,27.492111,21.885501,-3.273402,7.851163,25.195854
4,4.68858,4.320887,4.984527,0.184981,10.111343,4.504734,0.183846,7.851163,10.099175
...,...,...,...,...,...,...,...,...,...
95,16.806247,22.970269,5.57339,-1.73485,25.494867,19.888258,-3.082011,7.851163,23.373552
96,5.83882,6.489971,-5.375258,-0.052524,11.771005,6.164396,-0.325576,7.851163,11.453932
97,7.789262,10.091068,-42471.904214,-0.610034,14.546775,8.940165,-1.150903,7.851163,13.720649
98,13.478088,18.336784,-50952.692216,-1.255666,21.514045,15.907436,-2.429348,7.851163,19.774506


In [347]:
sim_value_changes.mean(axis=0)

Straddle           7.353008
SynLong            7.362989
CallSpread     -9382.310680
PutSpread         -0.071006
Stock             12.638441
Call               7.357998
Put               -0.004991
CoveredCall        7.015403
ProtectedPut      12.622878
dtype: float64

In [348]:
sim_value_changes.apply(lambda x:rml.calculateVar(x, 0), axis=0)

Straddle            0.663364
SynLong            10.625890
CallSpread      55854.213491
PutSpread           1.855309
Stock               1.802739
Call                5.795571
Put                 3.302657
CoveredCall        -2.247261
ProtectedPut       -1.764296
dtype: float64

In [349]:
sim_value_changes.apply(lambda x:rml.calculateES(x, 0), axis=0)

Straddle            0.904176
SynLong            13.874528
CallSpread      75458.951341
PutSpread           2.182845
Stock               6.297929
Call                6.636909
Put                 3.783170
CoveredCall         2.247929
ProtectedPut        0.662133
dtype: float64

In [350]:
S = 165
N = 25
current_date = datetime(2023, 3, 3)
div_date = datetime(2023, 3, 15)
r = 0.0425
div = 1

cal_amr_delta_num =  cal_partial_derivative(binomial_tree, 1, 'S0')

# Calculate the implied volatility for all portfolios
deltas = []
for i in range(len(portfolios.index)):
    if portfolios["Type"][i] == "Stock":
        deltas.append(1)
    else:
        option_type = portfolios["OptionType"][i]
        X = portfolios["Strike"][i]
        T = ((portfolios["ExpirationDate"][i] - current_date).days - 10) / 365
        div_time = int((div_date - current_date).days / (portfolios["ExpirationDate"][i] - current_date).days * N)
        delta = cal_amr_delta_num(option_type, S, X, T, div_time, div, sigma, r, N)
        deltas.append(delta)

# Store the deltas in portfolios
portfolios["deltas"] = deltas

alpha = 0.05
t = 10
result_dn = pd.DataFrame(index=sorted(portfolios['Portfolio'].unique()), columns=['Mean', 'VaR', 'ES'])
result_dn.index.name = 'Portfolio'
for pfl, df in portfolios.groupby('Portfolio'):
    gradient = S / df['CurrentValue'].sum() * (df['Holding'] * df['deltas']).sum()
    pfl_10d_std = abs(gradient) * std * np.sqrt(t)
    N = norm(0, 1)
    present_value = df['CurrentValue'].sum() 
    result_dn.loc[pfl]['Mean'] = 0
    result_dn.loc[pfl]['VaR'] = -present_value * N.ppf(alpha) * pfl_10d_std
    result_dn.loc[pfl]['ES'] = present_value * pfl_10d_std * N.pdf(N.ppf(alpha)) / alpha

result_dn
     

,Mean,VaR,ES
Portfolio,,,
Call,0,12.693933,15.918705
CallSpread,0,4.305794,5.399639
CoveredCall,0,4.755691,5.963828
ProtectedPut,0,12.649392,15.862847
Put,0,2.898182,3.634437
PutSpread,0,1.411042,1.769504
Stock,0,15.490022,19.425112
Straddle,0,9.795751,12.284267
SynLong,0,15.592115,19.553142


## Problem 3

In [118]:
def return_calculate(prices, method="discrete", dateColumn="Date"):
    vars_ = prices.columns
    nVars = len(vars_)
    vars_ = [var for var in vars_ if var != dateColumn]
    nVars = nVars - 1

    p = prices[vars_].to_numpy()
    n, m = p.shape
    p2 = np.empty((n-1, m))

    for i in range(n-1):
        for j in range(m):
            p2[i, j] = p[i+1, j] / p[i, j]

    if method.upper() == "discrete":
        p2 = p2 - 1.0
    elif method.upper() == "log":
        p2 = np.log(p2)
        
    dates = prices[dateColumn].iloc[1:n].to_numpy()
    res = pd.DataFrame({dateColumn: dates})
    
    for i in range(nVars):
        res[vars_[i]] = p2[:, i]

    return res


In [119]:
# data preparation
ff = pd.read_csv('F-F_Research_Data_Factors_daily.csv', parse_dates=['Date']).set_index('Date')
mom = pd.read_csv('F-F_Momentum_Factor_daily.csv', parse_dates=['Date']).set_index('Date')
# transfer percentage to value
factor = (ff.join(mom, how='right')/100).loc['2013-1-31':]

prices = pd.read_csv('DailyPrices.csv', parse_dates=['Date'])
all_returns = pd.DataFrame(return_calculate(prices)).set_index('Date')

stocks = ['AAPL', 'META', 'UNH', 'MA',  
          'MSFT' ,'NVDA', 'HD', 'PFE',  
          'AMZN' ,'BRK-B', 'PG', 'XOM',  
          'TSLA' ,'JPM' ,'V', 'DIS',  
          'GOOGL', 'JNJ', 'BAC', 'CSCO']
factors = ['Mkt-RF', 'SMB', 'HML', 'RF']
dataset = all_returns[stocks].join(data)
subset = dataset.dropna()

In [106]:
X = subset[factors]
X = sm.add_constant(X)

y = subset[stocks].sub(subset['RF'], axis=0)

betas = pd.DataFrame(index=stocks, columns=factors)
alphas = pd.DataFrame(index=stocks, columns=['Alpha'])


for stock in stocks:
    model = sm.OLS(y[stock], X).fit()
    betas.loc[stock] = model.params[factors]
    alphas.loc[stock] = model.params['const']

In [114]:
sub_return = pd.DataFrame(np.dot(factor[factors],betas.T), index=factor.index, columns=betas.index)
merge_return = pd.merge(sub_return,factor['RF'], left_index=True, right_index=True)
daily_expected_returns = merge_return.add(merge_return['RF'],axis=0).drop('RF',axis=1).add(alphas.T.loc['Alpha'], axis=1)

expected_annual_return = ((daily_expected_returns+1).cumprod().tail(1) ** (1/daily_expected_returns.shape[0]) - 1) * 252

expected_annual_return

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Date,,,,,,,,,,,,,,,,,,,,
2023-01-31,0.263543,-0.238417,0.492107,0.15057,0.228763,-0.056238,-0.00817,0.253431,0.041766,0.154862,0.017089,0.632155,0.433966,-0.128065,0.147108,-0.382507,0.006573,0.290983,-0.254746,0.026509


In [115]:
covariance_matrix = dataset[stocks].cov() * 252
covariance_matrix

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
AAPL,0.126877,0.139557,0.037447,0.081272,0.102937,0.171265,0.066193,0.032745,0.122117,0.055520,0.036945,0.037700,0.154880,0.058646,0.071406,0.087399,0.111906,0.022740,0.066245,0.066123
META,0.139557,0.400843,0.017102,0.102465,0.142255,0.240599,0.098845,0.045091,0.194794,0.061619,0.033637,0.020759,0.173121,0.073973,0.085022,0.127047,0.182074,0.021329,0.088791,0.076719
UNH,0.037447,0.017102,0.060922,0.031117,0.036318,0.046531,0.026045,0.032068,0.034737,0.028173,0.027861,0.026843,0.039128,0.033321,0.029590,0.022467,0.029806,0.022981,0.034744,0.028847
MA,0.081272,0.102465,0.031117,0.095762,0.079856,0.137369,0.056792,0.033440,0.096194,0.047520,0.031163,0.030837,0.097521,0.058309,0.082400,0.076987,0.079016,0.017382,0.063399,0.051734
MSFT,0.102937,0.142255,0.036318,0.079856,0.127839,0.175956,0.070916,0.035082,0.133856,0.052676,0.033859,0.031304,0.131911,0.056714,0.068230,0.088227,0.120259,0.020323,0.065226,0.060804
NVDA,0.171265,0.240599,0.046531,0.137369,0.175956,0.403814,0.112055,0.046362,0.221364,0.084290,0.041944,0.054455,0.291392,0.098399,0.118145,0.157837,0.188012,0.021758,0.113397,0.098663
HD,0.066193,0.098845,0.026045,0.056792,0.070916,0.112055,0.097074,0.033271,0.096833,0.042310,0.034414,0.015936,0.077754,0.043555,0.050164,0.063936,0.069626,0.022364,0.046487,0.048412
PFE,0.032745,0.045091,0.032068,0.033440,0.035082,0.046362,0.033271,0.070517,0.037274,0.031202,0.027470,0.019979,0.022283,0.031862,0.030650,0.024522,0.029355,0.027603,0.030812,0.029531
AMZN,0.122117,0.194794,0.034737,0.096194,0.133856,0.221364,0.096833,0.037274,0.242775,0.065772,0.030153,0.037446,0.187717,0.070819,0.083373,0.124253,0.149924,0.022616,0.085241,0.071419
BRK-B,0.055520,0.061619,0.028173,0.047520,0.052676,0.084290,0.042310,0.031202,0.065772,0.050175,0.024839,0.034026,0.061541,0.047076,0.042080,0.051383,0.056281,0.019263,0.050820,0.040421


In [116]:
def efficientOpt(returns, rf, covMat):
    if len(returns.shape) == 1:
        num_assets = returns.shape[0]
    else:
        num_assets = returns.shape[1]
    
    # Objective function to minimize
    def neg_sharpe_ratio(weights):
        port_return = np.sum(returns * weights)
        port_std_dev = np.sqrt(np.dot(weights.T, np.dot(covMat, weights)))
        sharpe_ratio = (port_return - rf) / port_std_dev
        return -sharpe_ratio
    
    # Constraints: weights sum to 1, all weights are non-negative
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
                   {'type': 'ineq', 'fun': lambda w: w}]
    
    # Bounds: weights must between 0 and 1
    bounds = [(0, 1) for _ in range(num_assets)]
    
    # Solve for optimal weights
    init_weights = np.ones(num_assets) / num_assets  # start with equal weights
    opt_result = minimize(neg_sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    
    # Return optimal weights and Sharpe ratio of resulting portfolio
    opt_weights = opt_result.x
    opt_port_return = np.sum(returns * opt_weights)
    opt_port_std_dev = np.sqrt(np.dot(opt_weights.T, np.dot(covMat, opt_weights)))
    opt_sharpe_ratio = (opt_port_return - rf_rate) / opt_port_std_dev
    
    return opt_weights*100, opt_sharpe_ratio

In [117]:
weights, sharpe_ratio = efficientOpt(expected_annual_return.values[0], 0.0425, covariance_matrix)

print("The Portfolio's Sharpe Ratio is: {:.2f}" .format(sharpe_ratio))

weights = pd.DataFrame(weights, index=expected_annual_return.columns, columns=['weight %']).round(2).T
weights

The Portfolio's Sharpe Ratio is: 2.26


,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
weight %,0.0,0.0,34.82,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.17,0.31,0.0,0.0,0.0,0.0,33.7,0.0,0.0
